In [1]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer
from timenorm_data_provider import TimeDataProvider
from transformers import RobertaForTokenClassification
from utils_ import *
import datasets

/Users/bulut/miniforge3/envs/timenorm/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Reusing dataset conll2003 (/Users/bulut/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee)
100%|████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 313.87it/s]


In [2]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)
data_provider = TimeDataProvider(corpus_dir="./example-data")
relation_to_extract = 'Sub-Interval'
distances = [
    '-6',
    '-5',
    '-4',
    '-3',
    '-2',
    '-1',
    'None',
    '1',
    '2',
    '3',
    '4',
]
f = open('./example-data/types.txt')
lines = f.readlines()
types = []
for line in lines:
    types.append(line.replace("\n", ""))
tokenized_dataset = data_provider.read_data_to_distance_format(tokenizer, relation_to_extract, distances, types)


Token indices sequence length is longer than the specified maximum sequence length for this model (1236 > 512). Running this sequence through the model will result in indexing errors


./example-data/Train/ID032_clinic_094/ID032_clinic_094
./example-data/Train/ID027_path_080/ID027_path_080
./example-data/Train/ID025_clinic_073/ID025_clinic_073
the entity that is linked does not exist in the document
0/116 is done
Skipping token types: 'ment [today] with' {'Calendar-Interval', 'This'}
Skipping token types: 's of [Friday], Sep' {'Intersection', 'Day-Of-Week'}
Skipping token types: 's of [Friday], Sep' {'Intersection', 'Day-Of-Week'}
Skipping token types: 'Rate=[20] /min' {'Frequency', 'Number'}
Skipping token types: 'Rate=[78] /min' {'Frequency', 'Number'}
Skipping token types: 'r to [today]).  A' {'Calendar-Interval', 'This'}
Skipping token types: 'that [today] or f' {'Calendar-Interval', 'This'}
Skipping token types: 'hing [tomorrow] morn' {'Calendar-Interval', 'Next', 'Intersection'}
Skipping token types: 'gery [tomorrow].  Sh' {'Calendar-Interval', 'Next'}
100/116 is done
Skipping token types: 's of [Wednesday], Apr' {'Intersection', 'Day-Of-Week'}
Skipping token t

In [3]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels_type', 'labels_distance'],
        num_rows: 116
    })
    validation: Dataset({
        features: ['input_ids', 'labels_type', 'labels_distance'],
        num_rows: 385
    })
    test: Dataset({
        features: ['input_ids', 'labels_type', 'labels_distance'],
        num_rows: 512
    })
})

In [26]:
train_dict = {"input_ids": input_ids, "labels_type": labels_type}
train_dataset = Dataset.from_dict(train_dict)


In [27]:
train_dataset

Dataset({
    features: ['input_ids', 'labels_type'],
    num_rows: 116
})

In [11]:
torch.tensor(train_dataset['input_ids']).shape

torch.Size([116, 118])

In [ ]:
temp_dataset = DatasetDict(dataset_dict)
temp_dataset

In [ ]:
temp_dataset['train']['a']

In [ ]:
task_name = 'pos'
from datasets import load_dataset

dataset = load_dataset("conll2003")

In [ ]:
tokenized_dataset = prepare_dataset(dataset, task_name)

In [ ]:
tokenized_dataset['train']['input_ids']

In [ ]:
shuffled_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(len(tokenized_dataset["train"])))
train_dataloader = DataLoader(shuffled_train_dataset, batch_size=train_batch_size)

In [5]:
train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=4)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

In [ ]:
tokenized_dataset['train']['labels'].shape

In [ ]:
tokenized_dataset['train']['input_ids'].shape

In [ ]:
label_list = dataset["train"].features[f"{task_name}_tags"].feature.names
num_labels = len(label_list)

In [6]:
model = RobertaForTokenClassification.from_pretrained(pretrained_model_name_or_path='roberta-base', 
                                                                num_labels=11)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able

In [7]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm

In [29]:
for batch in train_dataloader:
    model.train()
    batch = {k: torch.stack(v).to(device) for k, v in batch.items()}
    
    outputs = model.forward(input_ids=batch['input_ids'], labels=batch['labels_distance'])
    print(outputs)
    break

RuntimeError: expected scalar type Long but found Double

In [15]:
for batch in train_dataloader:
    print(batch['labels_type'][0])
    break

tensor([0., 0., 0., 0.], dtype=torch.float64)


In [25]:
batch['labels_type']

[tensor([0., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0.], dtype=torch.float64),
 tensor([4., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0.], dtype=torch.float64),
 tensor([5., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0.], dtype=torch.float64),
 tensor([2., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0.], dtype=torch.float64),
 tensor([4., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0.], dtype=torch.float64),
 tensor([5., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0.], dtype=torch.fl

In [24]:
torch.stack(batch['labels_type'])

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [4., 0., 0., 0.],
        [0., 0., 0., 0.],
        [5., 0., 0., 0.],
        [0., 0., 0., 0.],
        [2., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [4., 0., 0., 0.],
        [0., 0., 0., 0.],
        [5., 0., 0., 0.],
        [0., 0., 0., 0.],
        [2., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0.,

In [ ]:
from torch.nn import CrossEntropyLoss

In [ ]:
b_logits = outputs.logits
a_labels = batch['labels']
c_weights = torch.rand(a_labels.shape)

In [ ]:
a_labels = torch.reshape(a_labels, (-1,))
b_logits = torch.reshape(b_logits, (-1,b_logits.shape[-1]))
c_weights = c_weights.to(b_logits.dtype)
c_weights = torch.reshape(c_weights, (-1,))

In [ ]:
c_weights.size()

In [ ]:
b_logits.shape

In [ ]:
loss_fct = CrossEntropyLoss()

In [ ]:
loss_fct(b_logits, a_labels)

In [ ]:
loss_fct(b_logits, a_labels)*c_weights

In [ ]:
distances = [
    '-6',
    '-5',
    '-4',
    '-3',
    '-2',
    '-1',
    'None',
    '1',
    '2',
    '3',
    '4',
]

In [ ]:
weights = [0 if d=='None' else 500 for d in distances]


In [ ]:
weights

In [ ]:
num_epochs = 3
learning_rate = 2e-5
batch_size = 32
model_name = "roberta-base"
output_path = f"/xdisk/bethard/kbozler/timenorm/relation-extraction/method2/{model_name}/"

In [ ]:
output_path

In [ ]:
os.path.join(output_path, f"{num_epochs}epochs", f"{learning_rate}learning_rate", f"{batch_size}batch_size")

In [ ]:
features = datasets.Features({'a':list})